## Dataset and Paper

- Multi-Classification Problem, not binary.
- Merged the 12 datasets because for the task, they need to evaluate machine learning classifiers, not time-series models or sequential network behavior models. So they treated every flow (record) as independent (chronological event).
- Depending on the specific attack, some features are more important because of the design of the attack. But for the training part, they used all the features.

## Our task

#### New task

I think it is not useful to focus on classifying the attack. Instead, we can modify the task.

We can focus on:
  1. Classify the instantaneous intensity based on the volume.
  2. Predict near-term future traffic (volume/intensity).
No need to train the model to classify the type of attack, we can assume that an agent is doing that very well.

It can be related to the idea of Agentic system: 
- Agent 1: Detect the attack and categorize it (the work of the authors of the papers)
- Agent 2: Categorize the intensity (very basic because it is just a score)
- Agent 2 or 3: Predict the volume/intensity

Im some sense it is mandatory to overfit the network setup of the authors, because intensity 5 for this setup does not contain the same volume than intensity 5 for an other setup. However, in real life I think it's good to have an agentic system that is tailored to a specific network of a company. 

## Implementation

#### 2 technical possibilities

1. **Intensity classification + regression -> Per row:**
    - Classification: assign each flow/record into discrete intensity classes (e.g., Low / Medium / High) or quantile bins.
    - Regression: predict a continuous intensity score (packets/sec, bytes/sec, or normalized intensity).


2. **Time series -> Over time:**
    - Predict the future traffic (intensity) for the next few seconds or minutes. (e.g., total packets/sec).
    - Such as forecasting (regression) or future classification (e.g., “will intensity exceed threshold in next 10s?”).

In my opinion, the easier way is the 1. The setup could be less difficult. 

#### Technical Implementation

1. We can remove the labels of the attacks, because we are just interested about the intensity of the attack.
3. Create a new label (Depending on our choice): 
    - Intensity classification: Use bins: Low, medium, high or 1, 2, 3, 4, 5.
    - Intensity regression: Use score: Continuous intensity score
    - Time series: I need to do more research because I don't know the topic.
2. Train / Test models on dataset but independently to capture the evolution of the attack. But each dataset is not a specific attack anymore, it's just like a capture of network flow.

#### Label Intensity Score and bins

As a DDoS attack is when multiple compromised computers flood a target server or website with overwhelming traffic, so what matters to compute the intensity score is:
- How many packets per second (Flow Packets/s)
- How many bytes per second (Flow Bytes/s)
- How long the flow lasts (Flow Duration)


So there multiple possibilities to compute the Intensity score for each row.

For the Classification + Regression part, a very basic example of labels is: 
- Continous intensity = Packets per second (total_packets / duration).
- After that we create bins for the classification task.

A more robust approach will be to use multiple features and weight them.

#### Possible Feature Engineering

In our dataset, in my opinion, these are the most straightforward predictors of how "heavy" the traffic is:
- Flow Bytes/s
- Flow Packets/s
- Flow Duration (intensity relative to connection length)
- Fwd Packets/s
- Bwd Packets/s

We can maybe use the features for the Volume:
- Total Fwd Packets
- Total Backward Packets
- Total Length of Fwd Packets
- Total Length of Bwd Packet

And packet statistics: 
- Average Packet Size
- Packet Length Mean
- Fwd Packet Length Mean
- Bwd Packet Length Mean 

I tried to plot the Flow Bytes/s during the attack, and the Flow Bytes/s just for BENIGN time. The scale is different: 1e9 is 100 times larger than 1e7. so it illustrates that it can be good measure.